# Lab 03: Fun with `pandas`!

Below are some exercises to get you working with `pandas` to manipulate data. As always, get as far as you can, and ask for help when you need it! Your teacher (me), you instructor, and your classmates are all here to help each other get better at coding. Getting the code to work is important, but do also take the time to make sure you understand what the commands are doing. This time, (with the exception of the Stroop challenge), all I've given you is the code to download the data. Then you are on your own. For the Stroop challenge, I gave the you code for the first step—after that, it's up to you :-)

## Music sales challenge

Write a script that:

1. Combines the tables of best-selling physical singles and best-selling digital singles on the Wikipedia page "List_of_best-selling_singles"
2. Adds a column which marks whether each row is from the list of physical singles or digital singles
3. Outputs the artist and single name for the year you were born. If there is no entry for that year, take the closest year after you were born.
4. Outputs the artist and single name for the year you were 15 years old.

In [13]:
# Starter code...

#%pip install lxml
import pandas as pd

rawdata = pd.read_html("https://en.wikipedia.org/wiki/List_of_best-selling_singles")
# making a dataframe for physical and digital singles and reassigning the source column 
physical = pd.concat([rawdata[0],rawdata[1]])
digital = pd.concat([rawdata[3],rawdata[4]])
physical['Source'] = 'physical'
digital['Source'] = 'digital'
# Combining the two DataFrames using pd.concat
singles = pd.concat([physical, digital])
# Subsetting the data by requested years
birth = singles[singles['Released'].isin([1997, 2012])]
birth[['Artist', 'Single', 'Released']]

# Alternative way
# data from birth year
#birth = singles[singles['Released']==1997]
#birth[['Artist', 'Single']]
# data from year 15
#old = singles[singles['Released']==2012]
#old[['Artist', 'Single']]


,Artist,Single,Released
1,Elton John,"""Something About the Way You Look Tonight""/""Ca...",1997
9,Celine Dion,"""My Heart Will Go On""",1997
18,Imagine Dragons,"""Radioactive""",2012
7,Macklemore and Ryan Lewis featuring Wanz,"""Thrift Shop""",2012


## Space challenge

1. Make a single dataframe that combines the space missions from the 1950's to the 2020's
2. Write a script that returns the year with the most launches
3. Write a script that returns the most common month for launches
4. Write a script that ranks the months from most launches to fewest launches


In [52]:
# Starter code...

#%pip install lxml
import pandas as pd

rawdata = pd.read_html("https://en.wikipedia.org/wiki/Timeline_of_Solar_System_exploration")
# Concatinating the data and removing the reference column
space = pd.concat(rawdata[:8], ignore_index=True)
space.pop('Ref(s)')

# Converting the date column to datetime format
space['Launch date'] = pd.to_datetime(space['Launch date'], format='%d %B %Y')
# Extract the month and year into separate columns
space['Month'] = space['Launch date'].dt.month_name()  # Get full month name
space['Year'] = space['Launch date'].dt.year           # Extract the year
# finding the most common year and month, and displaying the months ranked
a = space['Year'].mode().tolist()
b = space['Month'].mode().tolist()
c = space['Month'].value_counts()
# Printing the requested info 
print(f'The year with the most launches is {a}', 
      f'\nThe most common month for launches is {b}', 
      f'\nAnd here are the months ranked:\n{c}')

The year with the most launches is [1965, 1967] 
The most common month for launches is ['November'] 
And here are the months ranked:
Month
November     30
August       27
September    22
October      22
July         21
January      19
December     19
May          17
March        15
February     14
June         14
April        13
Name: count, dtype: int64


## Supervillain challenge

1. Write a script that combines the tables showing supervillain debuts from the 30's through the 2010's
2. Write a script that ranks each decade in terms of how many supervillains debuted in that decade
3. Write a script that ranks the different comics companies in terms of how many supervillains they have, and display the results in a nice table (pandas dataframe)

In [95]:
rawdata = pd.read_html("https://en.wikipedia.org/wiki/List_of_comic_book_supervillain_debuts")

# Concatinating the data from the 30's through the 2010's and indicating the decade
villain = pd.concat(rawdata[3:12], keys=range(3, 12), ignore_index=False)
# Resetting the index to bring the keys into a column
villain = villain.reset_index(level=0)
# Renaming the new index column to something meaningful:)
villain = villain.rename(columns={'level_0': 'Decade'})

# Ranking both decades and companies and combining them into 1 df
decades = villain['Decade'].value_counts().reset_index()
company_counts = villain['Company'].value_counts().reset_index()
combined = pd.concat([decades,company_counts])
combined # the decades ranges from 3 (30's) to 11 (2010's)


,Decade,count,Company
0,6.0,228,NaN
1,7.0,97,NaN
2,8.0,92,NaN
3,9.0,84,NaN
4,10.0,49,NaN
5,4.0,47,NaN
6,5.0,26,NaN
7,11.0,9,NaN
8,3.0,4,NaN
0,NaN,338,DC


## Stroop challenge

Every year between 2015 and 2021, the students in my Language, Cognition, and the Brain course participated in a version of the Stroop task. Using a stopwatch (ok, using their phones), they recorded how fast they could say a list of things (either reading or naming colors or color words). The column names mean "Reading with No Interference", "Naming with Interference", "Naming with No Interference", and "Reading with Interference". The times are in seconds.

### Stroop challenge 1: 
Transform these data from wide format to long format, so that the result is a dataframe with
- 1 column named "Participant_id" with a unique number for each participant (you can use the row indices)
- 1 column named "Year" with the year data
- 1 column named "Task" that shows which task they were doing
- 1 column named "RT" that shows their response time

In [125]:
# Starter code 1...

df = pd.read_csv("https://raw.githubusercontent.com/ethanweed/Stroop/master/Stroop-raw-over-the-years.csv")
df.head()
# Creating the Participant_id column starting from 1
df['Participant_id'] = range(1, len(df)+1)
# Reshaping from wide to long format
df_long = pd.melt(
    df,
    id_vars=['Participant_id', 'Year'],  # Keep these columns
    value_vars=['Reading_NoInt', 'Naming_Int', 'Naming_NoInt', 'Reading_Int'],  # Columns to melt
    var_name='Task',  # Name of the new column that will hold the original column names
    value_name='RT'  # Name of the new column that will hold the values from the melted columns
)

#df_long

,Participant_id,Year,Task,RT
0,1,2015,Reading_NoInt,4.16
1,2,2015,Reading_NoInt,4.35
2,3,2015,Reading_NoInt,3.60
3,4,2015,Reading_NoInt,3.90
4,5,2015,Reading_NoInt,4.22
...,...,...,...,...
723,178,2021,Reading_Int,4.28
724,179,2021,Reading_Int,5.49
725,180,2021,Reading_Int,5.16
726,181,2021,Reading_Int,5.51


In [118]:
# Starter code 2...

# Make a new column using the dataframe indices as particpant numbers

#df.index.name = 'Participant_id'
#df = df.reset_index()

#df.reset_index(inplace = True)
# NOTE: This line does exactly the same thing as the line above:
# it replaces the original df with a new df with the updated index. That's what
# "inplace = True" means. Or, you can just assign the dataframe with the updated index
# to a new dataframe with the same name as the old dataframe, which is what I did above.
# The end result is the same.

#df

## Stroop challenge 2 (Advanced!!!):

Make a new dataframe which shows the mean response time (in seconds) for each task for each year.

In [126]:

df_long.groupby(['Year', 'Task'])['RT'].mean().reset_index()



,Year,Task,RT
0,2015,Naming_Int,8.617143
1,2015,Naming_NoInt,5.123571
2,2015,Reading_Int,4.446429
3,2015,Reading_NoInt,3.951429
4,2016,Naming_Int,8.859268
5,2016,Naming_NoInt,5.405610
6,2016,Reading_Int,5.340000
7,2016,Reading_NoInt,4.076098
8,2017,Naming_Int,9.311765
9,2017,Naming_NoInt,5.771176
